### 코드 전체 흐름
1. 문서의 내용을 읽는다
2. 문서를 쪼갠다.
- 토큰수 초과로 답변을 생성하지 못할 수 있고
- 문서가 길면(Input이 길면) 답변 생성이 오래걸림
3. 임베딩 -> 벡터 DB에 저장
4. 질문이 있을 때, 벡터 DB에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [1]:
%pip install --upgrade --quiet  docx2txt langchain-community
%pip install -qU langchain-text-splitters
%pip install -U langchain langchainhub --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,    # 문서를 쪼갤 때 하나의 chunk 가 가지는 토큰 수
    chunk_overlap=200,  # 텍스트를 분할할 때, 각 chunk(조각) 사이에 중복되는 부분의 토큰 수
)

loader = Docx2txtLoader('./tax_with_table.docx')
document = loader.load()                                           #단순히 문서를 읽을 때
document_list = loader.load_and_split(text_splitter=text_splitter) # 문서를 쪼갤 때

In [3]:
print('쪼개기 전 문서 : ',len(document))
print('쪼갠 문서 : ',len(document_list))

쪼개기 전 문서 :  1
쪼갠 문서 :  225


In [4]:
from dotenv import load_dotenv                # OpenAI API Key 사용을 위해 환경변수 등록
from langchain_openai import OpenAIEmbeddings # OpenAI 의 Embedding을 사용

load_dotenv() # 환경변수 불러오기

embeddings = OpenAIEmbeddings(model='text-embedding-3-large') # 기본모델은 002인 예전모델이다. 신규 모델을 사용하기 위해 추가

In [5]:
%pip install -qU langchain-pinecone pinecone-notebooks

Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip uninstall -y pinecone-notebooks

Found existing installation: pinecone-notebooks 0.1.1
Uninstalling pinecone-notebooks-0.1.1:
  Successfully uninstalled pinecone-notebooks-0.1.1
Note: you may need to restart the kernel to use updated packages.


In [7]:
import os

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'tax-table-index'
pinecone_api_key = os.environ.get('PINECONE_API_KEY')

pc = Pinecone(api_key=pinecone_api_key)
database = PineconeVectorStore.from_documents(document_list, embeddings, index_name=index_name)

C:\Users\DataLogics\miniconda3\envs\llm_lab\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [8]:
query = '연봉 3000만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query, 1); # similarity_search() 함수는 유사도 검색을 위한 함수이다.

In [9]:
retrieved_docs

[]

In [10]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [11]:
# 기존에는 질문만 날렸지만 이제는 질문+문서 를 질의한다. 즉, 프롬프트 작성
prompt = f"""[Identity]
- 당신은 현존하는 세계 최고의 한국 소득세 전문가 입니다
- [Context]를 참고해서 사용자의 질문에 답변해주세요

[Context]
{retrieved_docs}

Question: {query}
"""


In [12]:
ai_message = llm.invoke(prompt)

In [13]:
ai_message.content

'연봉 3,000만 원인 직장인의 소득세를 계산하기 위해서는 기본적으로 소득세율과 공제 항목들을 고려해야 합니다. 한국의 소득세는 누진세율 구조를 가지며, 소득 구간에 따라 세율이 다르게 적용됩니다.\n\n2023년 현재 한국의 근로소득세율은 다음과 같습니다:\n\n- 1,200만 원 이하: 6%\n- 1,200만 원 초과 ~ 4,600만 원 이하: 15%\n- 4,600만 원 초과 ~ 8,800만 원 이하: 24%\n- 8,800만 원 초과 ~ 1억 5천만 원 이하: 35%\n- 1억 5천만 원 초과 ~ 3억 원 이하: 38%\n- 3억 원 초과 ~ 5억 원 이하: 40%\n- 5억 원 초과 ~ 10억 원 이하: 42%\n- 10억 원 초과: 45%\n\n연봉 3,000만 원에 대한 소득세를 계산해보면:\n\n1. 1,200만 원까지는 6% 세율 적용:\n   - 1,200만 원 × 6% = 72만 원\n\n2. 1,200만 원 초과분 1,800만 원(3,000만 원 - 1,200만 원)에는 15% 세율 적용:\n   - 1,800만 원 × 15% = 270만 원\n\n따라서 총 소득세는 72만 원 + 270만 원 = 342만 원입니다.\n\n이 금액은 기본적인 계산으로, 실제 납부 세액은 각종 공제(예: 인적 공제, 특별 공제 등)를 반영하여 달라질 수 있습니다. 공제 항목들은 개인의 상황에 따라 다양하게 적용될 수 있으므로, 정확한 세금 계산을 위해서는 개별적인 공제 사항들을 고려해야 합니다.'

In [14]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

C:\Users\DataLogics\miniconda3\envs\llm_lab\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [15]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [16]:
# QA Chain 만들기
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt":prompt}
)

In [17]:
ai_message = qa_chain({"query" : query})

C:\Users\DataLogics\AppData\Local\Temp\ipykernel_19444\914447175.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ai_message = qa_chain({"query" : query})


In [18]:
ai_message

{'query': '연봉 3000만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 3000만원인 직장인의 소득세는 과세표준 1400만원 초과 5000만원 이하 구간에 해당하므로, 84만원 + (1400만원을 초과하는 금액의 15퍼센트)로 계산됩니다. 따라서, 소득세는 약 384만원입니다.'}